## 通過 Google Cloud 進行驗證

In [1]:
import os

# 請確保 'client_secret.json' 檔案存在於目前目錄或指定路徑
client_secret_file = "client_secret.json"

if not os.path.exists(client_secret_file):
    raise FileNotFoundError(f"client_secret.json 文件不在路徑中：{client_secret_file}")

# 執行 gcloud 登入
os.system(
    f"gcloud auth application-default login --client-id-file={client_secret_file} --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'"
)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=826653416557-5f254qj34fu2kir6pl0bqin5jf8uifaf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=eagOGqqQH373iJ0iRgg09EsKMio0sK&access_type=offline&code_challenge=EuOZKNPVJd7Te5VyBm_rOMRT2AtOMFFpD2XKerHFo00&code_challenge_method=S256


Credentials saved to file: [/Users/samhsiao/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).


0

## 透過自動化腳本進行驗證

單獨生成哈希

In [2]:
import base64
import hashlib
import os

# 產生隨機的 code_verifier
code_verifier = base64.urlsafe_b64encode(os.urandom(32)).decode("utf-8").rstrip("=")

# 使用 SHA256 對 code_verifier 進行哈希
code_challenge = (
    base64.urlsafe_b64encode(hashlib.sha256(code_verifier.encode("utf-8")).digest())
    .decode("utf-8")
    .rstrip("=")
)

print(f"code_verifier: {code_verifier}")
print(f"code_challenge: {code_challenge}")

code_verifier: cdqQf_I_aYYPq2rn2LOPatirD6-4r22hNZF1yb_nd0I
code_challenge: JTUYVbeJ8avpdo0612wnEbKEWr2M4WTMI8-KAe9whOc


完整自動化腳本

In [3]:
import base64
import hashlib
import os
import json


def generate_code_verifier(length=64):
    """ 產生一個隨機的 code_verifier"""
    if length < 43 or length > 128:
        raise ValueError("code_verifier 長度必須在 43 至 128 位元組之間")
    code_verifier = (
        base64.urlsafe_b64encode(os.urandom(length)).decode("utf-8").rstrip("=")
    )
    return code_verifier


def generate_code_challenge(code_verifier):
    """使用 SHA-256 對 code_verifier 進行哈希，然後進行 Base64 URL 安全編碼"""
    code_challenge = (
        base64.urlsafe_b64encode(hashlib.sha256(code_verifier.encode("utf-8")).digest())
        .decode("utf-8")
        .rstrip("=")
    )
    return code_challenge


# 讀取 client_secret.json 文件
with open("client_secret.json", "r") as file:
    client_info = json.load(file)

# 獲取 client_id
client_id = client_info["installed"]["client_id"]

# 生成 code_verifier 和 code_challenge
code_verifier = generate_code_verifier()
code_challenge = generate_code_challenge(code_verifier)

# 生成 gcloud 命令
gcloud_command = (
    f'gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth'
    f"?response_type=code"
    f"&client_id={client_id}"
    f"&scope=https://www.googleapis.com/auth/cloud-platform+https://www.googleapis.com/auth/generative-language.tuning"
    f"&state=12345"
    f"&access_type=offline"
    f"&code_challenge={code_challenge}"
    f"&code_challenge_method=S256"
    f"&redirect_uri=http://localhost"
    f'&token_usage=remote"'
)

# 輸出 gcloud 命令
# print("Generated gcloud command:")
print(gcloud_command)

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=826653416557-5f254qj34fu2kir6pl0bqin5jf8uifaf.apps.googleusercontent.com&scope=https://www.googleapis.com/auth/cloud-platform+https://www.googleapis.com/auth/generative-language.tuning&state=12345&access_type=offline&code_challenge=z7nSEKV37XaNla5c-4578mFBheicq5ZtV4tiYZpU4cM&code_challenge_method=S256&redirect_uri=http://localhost&token_usage=remote"


### 使用腳本設置環境變數並查詢

In [4]:
import os
import json

# 讀取 client_secret.json 文件，並提取 project_id
with open('client_secret.json', 'r') as file:
    data = json.load(file)
    project_id = data['installed']['project_id']

# 獲取訪問令牌
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

# 設置環境變數
os.environ['access_token'] = access_token
os.environ['project_id'] = project_id
os.environ['base_url'] = "https://generativelanguage.googleapis.com"

# 驗證設置是否成功
print("Access Token:", os.environ['access_token'])
print("Project ID:", os.environ['project_id'])
print("Base URL:", os.environ['base_url'])

Access Token: ya29.a0AXooCguqk7rzodAK9OdlRnz-67vWW4_u2KAkAvBsxaGftEmgbHJSo4r6H0cMI2aqqNGJKAFPrQTu5VqTdc7Jl5-qeuoxOB-CfIcdk3KmgeAugLMzQk_TJaeMcGBOgdcg8V9Ymk2hfvZ3cubVFxO5MC2vqxnnH3exqNwsaCgYKAZMSARASFQHGX2MieRB_yOCZCdEK9vnlTxu6aA0171
Project ID: myproject-20240622
Base URL: https://generativelanguage.googleapis.com


#### 僅查詢環境變數

In [5]:
# 驗證設置是否成功
print("Access Token:", os.environ['access_token'])
print("Project ID:", os.environ['project_id'])
print("Base URL:", os.environ['base_url'])

Access Token: ya29.a0AXooCguqk7rzodAK9OdlRnz-67vWW4_u2KAkAvBsxaGftEmgbHJSo4r6H0cMI2aqqNGJKAFPrQTu5VqTdc7Jl5-qeuoxOB-CfIcdk3KmgeAugLMzQk_TJaeMcGBOgdcg8V9Ymk2hfvZ3cubVFxO5MC2vqxnnH3exqNwsaCgYKAZMSARASFQHGX2MieRB_yOCZCdEK9vnlTxu6aA0171
Project ID: myproject-20240622
Base URL: https://generativelanguage.googleapis.com


In [ ]:
import json

# 解析 JSON 文件
first_page = json.load(open('tunemodel.json'))
os.environ['modelname'] = first_page['metadata']['tunedModel']

print(os.environ['modelname'])